# Machine Learning - Project 2:
# _Autoencoder for mathematical modeling of blood flow in a stenosis context_

## Context

In this project, we are going to analyze data derived from...

Our goals:
1. modeliser mathématiquement l’écoulement sanguin à l’aide de PDE dépendante de 2-3 paramètres physique
2. simuler l’écoulement par un code d’éléments fini ou similaire
3. générer beaucoup de solutions avec une grande nombre de paramètres différents. (Les 2-3, pris de façon aléatoire)
4. a. utiliser les solutions numérique pour établir un auto encoder qui au milieu n’ai que 2-5 hyper-paramètres libres  
b. quel erreur on obtient ? Est-ce possible de réduire le nombre d’hyper-paramètres ?
5. étudier s’il y a une rélation entre les 2-5 hyper-paramètre et les paramètres physique  
a. à l’aide de statistiques
5. b. à l’aide d’un DNN (différent de 4a)
6. (optionnel) faire un DNN entre l’input de 4a et output les paramêtres physique. Et/ou l’inverse.
7. Discussion et conclusions 

Abbreviations used:
- $N_u$ = total number of spatial points per simulation
- $N_t$ = total number of time steps per simulation
- $N_s$ = total number of simulations

## Table of contents

[1. Data exploration](#data_exploration) 
- [Imports](#1imports)
- [Pathways](#1pathways)
- [Loading](#1load)
- [Exploration](#1exploration)

[2. Data preprocessing](#preprocessing)

[3. Autoencode](#classifier)

## 1. Data exploration  <a name="data_exploration"></a>

### Imports  <a name="1imports"></a>

In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from helpers import *
from preprocessing import *
from sklearn.model_selection import train_test_split
%load_ext autoreload
%autoreload 2

### Pathways <a name="1pathways"></a>

In [2]:
DATA_PATH = 'data/'
DATA_Ux_PATH = DATA_PATH + 'u1_small.csv.bz2'
DATA_Uy_PATH = DATA_PATH + 'u2_small.csv.bz2'

### Loading <a name="1load"></a>

Let's load the data which consists of 2 matrices Ux, Uy denoting the x and y coordinates of the speed.

In [ ]:
# Loading
Ux_pd = pd.read_csv(DATA_Ux_PATH, header=None)
Uy_pd = pd.read_csv(DATA_Uy_PATH, header=None)
# Converting from dataframe to numpy
Ux = Ux_pd.to_numpy()
Uy = Uy_pd.to_numpy()

### Exploration <a name="1exploration"></a>

Let's have a closer look at our imported data sets. The columns represent the time steps and the rows represent a point of our blood vessel mesh and each 5509 row a new simulation.

In [5]:
print("Our input array Ux is of shape:",Ux.shape)
print("Our input array Uy is of shape:",Uy.shape)
print("Printing a row of Ux:", '\n', Ux[10,:], '\n')
print("Printing a row of Uy:", '\n', Uy[10,:], '\n')

Our input array Ux is of shape: (3029950, 110)
Our input array Uy is of shape: (3035459, 110)
Printing a row of Ux: 
 [0.06417897 0.25470302 0.55885671 0.9493     1.38895813 1.83536662
 2.24512906 2.57829978 2.80235851 2.89542637 2.99010086 3.07846599
 3.16237576 3.24272178 3.31993687 3.39424293 3.46575844 3.53454851
 3.60064961 3.66408252 3.72485926 3.7829869  3.83846969 3.89131023
 3.94151015 3.98907053 4.03399206 4.07627523 4.11592037 4.15292772
 4.18729746 4.21902971 4.24812457 4.27458211 4.29840238 4.31958542
 4.33813127 4.35403995 4.36731148 4.37794586 4.38594311 4.39130324
 4.39402626 4.39450555 4.39404578 4.39284058 4.39093435 4.38834725
 4.38508917 4.38116522 4.37657817 4.37132956 4.36542029 4.35885087
 4.3516216  4.34373269 4.33518425 4.32597635 4.31610906 4.3055824
 4.29439639 4.28255106 4.27004643 4.25688248 4.24305925 4.22857672
 4.2134349  4.19763381 4.18117343 4.16405377 4.14627483 4.12783661
 4.10873912 4.08898235 4.06856631 4.04749099 4.02575639 4.00336252
 3.98030938 

For the rest of the analysis, we need to figure out the number of simulation step. As we know that we previsouly generated 25 simulations on Matlab and all the new simulations are appended row-wise, we can deduce it with the following computation.

In [6]:
size = Ux.shape[0]/25
print("We have run", Ux.shape[0],"simulations with a step of",size)

We have run 3029950 simulations with a step of 121198.0


## 2. Data preprocessing  <a name="preprocessing"></a>

Let's check if we have any NaN or None values in our dataframe.

In [7]:
print(np.count_nonzero(np.isnan(Ux)))
print(np.count_nonzero(np.isnan(Uy)))

0
0


It seems there are no None values so we can start direclty preprocess our datasets.

ADDITIONAL IDEAS FOR PREPROCESSING
- remove columns with 0 std dev?
- standardization?

Let's sample our data points into the following ratios : 

In [8]:
ratio_pts = 0.1
ratio_time = 1

new_Ux, new_Uy = sample(Ux, Uy, ratio_pts, ratio_time)
print(new_Ux.shape, new_Uy.shape )

(302500, 110) (302500, 110)


In [9]:
new_Nu, new_Nt = get_Nu_Nt_sampled(Ux, new_Ux)

print('With the sampling we got from ', 5509, ' positions to ', new_Nu, ' positions')
print('With the sampling we got from ', 110, ' time steps to ', new_Nt, ' time steps')

With the sampling we got from  5509  positions to  550  positions
With the sampling we got from  110  time steps to  110  time steps


Let's flatten our matrices into a single matrix with dimensions $((2 N_u N_t), N_s)$

In [10]:
print(new_Ux)

[[2.4669628  2.95609394 2.9131648  ... 2.16528591 2.02004071 2.93534615]
 [0.32461604 0.39327742 0.38658752 ... 0.29807351 0.26373287 0.39251852]
 [2.23956366 2.68549092 2.64593838 ... 1.97405694 1.83301461 2.66750225]
 ...
 [3.58186483 4.27567226 4.21769347 ... 3.08544866 2.94060993 4.2379438 ]
 [0.16419236 0.20173798 0.19768457 ... 0.15929658 0.13202943 0.2026274 ]
 [1.07779084 1.29176932 1.27313156 ... 0.9428681  0.88227332 1.28281555]]


In [11]:
flattened_array = flatten(new_Ux, new_Uy, ratio_pts)
flattened_array.shape

(550, 121000)

Let's make a sanity check that the dimension of 1 datapoint is indeed ${2 * new_{N_t} * new_{N_u})}$

In [12]:
assert flattened_array.shape[1] == 2*new_Nt*new_Nu

print(2*new_Nt*new_Nu)

121000


## 3. Auto-encoder  <a name="autoencoder"></a>

We split the data set into a training a testing set to be able to evaluate our autoencoder.

In [13]:
# Set seed 
seed = 123
flattened_array_train, flattened_array_test = train_test_split(flattened_array, test_size=0.1, random_state=seed)
y_train, y_test = flattened_array_train, flattened_array_test

print(flattened_array_train.shape)
print(flattened_array_test.shape)

(495, 121000)
(55, 121000)


In [14]:
from autoencoder import *
Autoencoder.train(flattened_array_train)

epoch : 1/100, loss = 58.193390
epoch : 2/100, loss = 58.184418
epoch : 3/100, loss = 58.206425
epoch : 4/100, loss = 58.114281
epoch : 5/100, loss = 58.176647
epoch : 6/100, loss = 58.243023
epoch : 7/100, loss = 58.234535
epoch : 8/100, loss = 58.185444
epoch : 9/100, loss = 58.164902
epoch : 10/100, loss = 58.153805
epoch : 11/100, loss = 58.201366
epoch : 12/100, loss = 58.186378
epoch : 13/100, loss = 58.176662
epoch : 14/100, loss = 58.113903
epoch : 15/100, loss = 58.176727
epoch : 16/100, loss = 58.233944
epoch : 17/100, loss = 58.151752
epoch : 18/100, loss = 58.188232
epoch : 19/100, loss = 58.223103
epoch : 20/100, loss = 58.189640
epoch : 21/100, loss = 58.203979
epoch : 22/100, loss = 58.223717
epoch : 23/100, loss = 58.114792
epoch : 24/100, loss = 58.167294
epoch : 25/100, loss = 58.205078
epoch : 26/100, loss = 58.146725
epoch : 27/100, loss = 58.180946
epoch : 28/100, loss = 58.202721
epoch : 29/100, loss = 58.193977
epoch : 30/100, loss = 58.155376
epoch : 31/100, los

## 4. Passer en 2D 

## 5. Hyper-parameters and physics parameters relationship

## 6. Discussion & conclusion